#### Notes
Need to run `pip install PyPaperBot` first, if the Python module is not yet installed.

See https://github.com/ferru97/PyPaperBot for more information about the module PythonPaperBot.

This script works in Windows. To run in Linux, change the line `process=subprocess.Popen(["powershell.exe","-Command",command],stdout=subprocess.PIPE)` and switch out powershell.exe for the Linux executables.

#### Instruction

- Create an empty folder and put this Jupyter Notebook in it. Make sure the folder path is not too long (if working on a network drive with very long path: mount the path as net drive first to shorten the path).

- Put the keywords you want to search for in Keywords\_\*.txt files. You can create multiple Keywords\_\*.txt files. The python script will generate all possible combinations of keywords and store them in a text file "Search_Terms.txt"

- Change the variable `download_number_per_keyword` and `year_from` according to your preference.
- Run the "Download papers" section. The script will create a new folder for each search terms and store the downloaded paper inside.

#### Get a tor session
This part does not work as tor is apparently blocked by my ISP. If you can get a tor proxy, you can add proxy settings in the PyPyperBot argument.

In [11]:
import requests
from tor_proxy import tor_proxy
port=tor_proxy()
session = requests.session()
# Tor uses the 9050 port as the default socks port
session.proxies = {'http':  'socks5://127.0.0.1:'+port,
                    'https': 'socks5://127.0.0.1:'+port}
print(session.get("http://httpbin.org/ip").text+':'+port)

AttributeError: 'tuple' object has no attribute 'tb_frame'

#### Download papers

In [1]:
import os
import itertools
import glob
import pandas
download_number_per_keyword=120 # Try to download the first so many search results. Each page of Google Scholar contains 10 search results, so 80 will mean downloading all papers shown in the first 8 pages.
year_from=1956 # Limitation of how old the paper can be
KeywordsListFiles=glob.glob("../1_Keywords/Keywords_*.txt")
Keywords=[[]]*len(KeywordsListFiles)
for i in range(len(KeywordsListFiles)):
    with open(KeywordsListFiles[i],'r',encoding='utf-8') as f:
        keyw=f.read().splitlines()
        Keywords[i]=keyw
    i+=1
Keywords_Combinations=list(itertools.product(*Keywords))
Keywords_Combinations.reverse() # Make sure the latest added keywords are searched first
with open("../1_Keywords/Search_Terms_Combinations.txt",'w',encoding='utf-8') as f:
    f.write('\n'.join([' '.join(x) for x in Keywords_Combinations]))
import subprocess
import sys
# ________Get a tor proxy_________
# from datetime import datetime
# from datetime import timedelta
# from dateutil.relativedelta import relativedelta
# last_time_proxy_change = datetime.now() - relativedelta(years=1) # Put the first time stamp one year ago
# _______Read search terms________
Search_Terms=[]
if os.path.isfile("../1_Keywords/Search_Terms_Special.txt"):
    with open("../1_Keywords/Search_Terms_Special.txt",'r',encoding='utf-8') as f:
        Search_Terms=Search_Terms+f.read().splitlines()
with open("../1_Keywords/Search_Terms_Combinations.txt",'r',encoding='utf-8') as f:
    Search_Terms=Search_Terms+f.read().splitlines()
for combi in Search_Terms:
    # if datetime.now() - last_time_proxy_change > timedelta(hours=8):
    #     tor_port=tor_proxy() # Get a new proxy 
    #     last_time_proxy_change=datetime.now()
    already_downloaded=False
    try:
        os.mkdir('../2_Papers/'+combi)
    except FileExistsError:
        try:
            Downloaded_Papers=pandas.read_csv("../2_Papers/"+combi+"/result.csv",header=0)
            if Downloaded_Papers["Downloaded"].sum() > download_number_per_keyword * 0.5:
                already_downloaded=True
        except FileNotFoundError:
            print("Folder \""+combi+"\" is created but empty.")
    if already_downloaded:
        print("There's already some documents downloaded for the title: "+combi+". Skipping")
    if not already_downloaded:
        command="python.exe -m PyPaperBot --query=\""+combi+"\" --scholar-pages="+str(round(download_number_per_keyword/10))+"  --min-year="+str(year_from)+" --dwn-dir=\"..\\2_Papers\\"+combi+"\""
        # command="for($i=1;$i -lt 3; $i++) {Write-Host \"$($i) "+combi+"\"; Start-Sleep -s 1}" # This is a test command to see if the following subprocess call works.
        print("Searching for \""+combi+"\" and downloading...")
        process=subprocess.Popen(["powershell.exe","-Command",command],stdout=subprocess.PIPE)
        for c in iter(lambda: process.stdout.read(1), b''):
            sys.stdout.write(c.decode('utf-8','ignore'))
import glob
for f in glob.glob('../2_Papers/**/(?)*.pdf',recursive=True):
    os.remove(f) # remove files with (2) in the beginning
    print("\"",f,"\"removed.")

There's already some documents downloaded for the title: Resistance spot welding of pressure die-cast aluminium. Skipping
There's already some documents downloaded for the title: Tungsten inert gas welding Nickel Aerospace applications. Skipping
There's already some documents downloaded for the title: Tungsten inert gas welding Nickel Lightweight applications. Skipping
There's already some documents downloaded for the title: Tungsten inert gas welding Nickel Automotive applications. Skipping
There's already some documents downloaded for the title: Tungsten inert gas welding Magnesium Aerospace applications. Skipping
Folder "Tungsten inert gas welding Magnesium Lightweight applications" is created but empty.
Searching for "Tungsten inert gas welding Magnesium Lightweight applications" and downloading...
PyPaperBot is a Python tool for downloading scientific papers using Google Scholar, Crossref and SciHub.
If you like this project, you can give me a cup of coffee at --> https://www.payp

#### Check repeated files and remove duplicated files

This part will create a file called Duplication.json

If the "Download paper" session was interrupted while downloading and then run again, the already downloaded paper will be downloaded again with "(2)" added to the beginning of file name. The following script assumes that all files with "(2)" in its name is a duplicate and will delete it.

In [11]:
import glob
import os
for f in glob.glob('../2_Papers/**/(?)*.pdf',recursive=True):
    os.remove(f) # remove files with (2) in the beginning
    print("\"",f,"\"removed.")
import pandas
import json
Downloaded_PDFs={}
# for f in glob.glob('./**/*.pdf',recursive=True):
#     Downloaded_PDFs[]
for (dirpath, dirnames, filenames) in os.walk('../2_Papers'):
    PDFs=[x for x in filenames if x.endswith('.pdf')]
    Downloaded_PDFs[os.path.basename(dirpath)]=PDFs # This is a dictionary
Downloaded_Keywords=Downloaded_PDFs.keys() # this is essentially the same as the variable "combi" above, but may have different sorting. So here another variable. 
Downloaded_PDFs_List=Downloaded_PDFs.values() # this is a list of PDF file names
Downloaded_PDFs_List_Flat=[
    x
    for xs in Downloaded_PDFs_List
    for x in xs
] # convert a list of lists to a list of elements (remove the sub-level list)
Downloaded_PDFs_List_Flat_LowerCase=[x.lower() for x in Downloaded_PDFs_List_Flat]
Downloaded_PDF_List_NoDuplicate_LowerCase=list(dict.fromkeys(Downloaded_PDFs_List_Flat_LowerCase))
# Downloaded_PDF_List_NoDuplicate=list(dict.fromkeys(Downloaded_PDFs_List_Flat)) # remove all duplications. Commented out because this is case sensitive.
Downloaded_PDF_List_NoDuplicate=[]
for i in range(1,len(Downloaded_PDFs_List_Flat_LowerCase)):
    pass
PDF_Repitition_Count=[1]*len(Downloaded_PDFs_List_Flat)
PDF_File_Already_Appeared=[False]*len(Downloaded_PDFs_List_Flat)
PDF_Repitition_List={}
for i in range(len(Downloaded_PDFs_List_Flat)):
    PDF_Repitition_Count[i]=Downloaded_PDFs_List_Flat.count(Downloaded_PDFs_List_Flat[i])
    if PDF_Repitition_Count[i] > 1:
        if Downloaded_PDFs_List_Flat[i] in Downloaded_PDFs_List_Flat[:i]:
            PDF_File_Already_Appeared[i]=True
        if not PDF_File_Already_Appeared[i]:
            PDF_Appear_In=[]
            for key in Downloaded_PDFs:
                if Downloaded_PDFs_List_Flat[i] in Downloaded_PDFs[key]:
                    PDF_Appear_In.append(key)
            PDF_Repitition_List[Downloaded_PDFs_List_Flat[i]]=PDF_Appear_In
print(str(len(Downloaded_PDFs_List_Flat)),"PDFs downloaded. Unique PDF files (by name):",str(len(Downloaded_PDF_List_NoDuplicate)))
with open("../4_Temp/Unique_PDF_files.txt","w",encoding="utf-8") as f:
    f.write('\n'.join(Downloaded_PDF_List_NoDuplicate))
with open("../4_Temp/Total_Numbers_unique_PDF_files.txt","w",encoding="utf-8") as f:
    f.write(str(len(Downloaded_PDF_List_NoDuplicate)))
# PDF_Repitition_Table=pandas.DataFrame(dict([(key, pandas.Series(value)) for key, value in PDF_Repitition_List.items()]))
# import json
# with open("../4_Temp/Duplication.json","w",encoding="utf-8") as fp:
#     json.dump(PDF_Repitition_List,fp)
# with open("../4_Temp/Duplication.json",'r',encoding="utf-8") as f:
#     PDF_Repitition_List=json.load(f)
for key in PDF_Repitition_List:
    for i in range(len(PDF_Repitition_List[key])):
        if i !=0:
            Folder_Name=PDF_Repitition_List[key][i]
            File_Name=key.split(".pdf")[0]+".pdf"
            try:
                os.remove('../2_Papers/'+Folder_Name+'/'+File_Name)
                print('Duplicated PDF file "'+Folder_Name+'\\'+File_Name+'" removed.')
            except FileNotFoundError:
                print('Duplicated PDF file "'+Folder_Name+'\\'+File_Name+'" was already removed.')
with open('../4_Temp/PDF_files_with_FullPath.txt','w',encoding='utf-8') as f:
    PDFs_WFullPath=[]
    for F in glob.glob('../2_Papers/**/*.pdf',recursive=True):
        PDFs_WFullPath.append(F)
    f.write('\n'.join(PDFs_WFullPath))

3350 PDFs downloaded. Unique PDF files (by name): 3350


#### List papers by DOI

This part will check the `result.csv` file from each subfolder and make a list (pandas DataFrame) of all downloaded papers.

If paper download of one specific search term is interrupted, the download will be incomplete and no `result.csv` or `bibtex.bib` file will be created. The folder will be skipped.

The list (DataFrame) will be stored in `List_Papers.json` and will look like this:
```
{
        "Name": "[Title of the paper]",
        "DOI": "[DOI of the paper]",
        "Year": [Year in int],
        "Journal": "[Journal Name]",
        "Search terms": [
            "List",
            "of",
            "Search",
            "Terms",
            "where",
            "this",
            "paper",
            "appeared"
        ]
}
```

In [10]:
import os,glob,pandas,json
Columns_in_CSV=['Name','DOI','Year','Journal','PDF Name','Bibtex'] #'Scholar Link','Authors'
List_Papers=pandas.DataFrame(columns=Columns_in_CSV+['Search term']) # Initialized DataFrame format: {"doi":[], "title":[], "searchterms":[] }
for res in glob.glob('../2_Papers/**/result.csv',recursive=True):
    search_term=os.path.basename(os.path.dirname(res))
    # print("Reading result.csv of "+dirpath)
    search_results=pandas.read_csv(res,header=0)
    search_results.drop(columns=search_results.columns.difference(Columns_in_CSV+['Downloaded']),inplace=True)
    # search_results=search_results.loc[search_results.Downloaded,:] # Remove not downloaded entries (where "Downloaded=False")
    # search_results.drop(columns=['Downloaded'],inplace=True) 
    search_results["Search term"]=[search_term]*len(search_results)
    List_Papers=pandas.concat([List_Papers,search_results])
List_Papers.fillna("Unknown",inplace=True)
with open("../4_Temp/Unique_PDF_files.txt","r",encoding="utf-8") as f:
    PDFs=f.read().splitlines()
with open("../4_Temp/PDF_files_with_FullPath.txt","r",encoding="utf-8") as f:
    PDFs_WFullPath=f.read().splitlines()
print("After investigations, it seems that some papers are downloaded but their download status was still \"false\"\nTry to parse those PDF files...")
import re # regex
for i in range(len(List_Papers.index)):
    if not ((List_Papers.iloc[i])["Downloaded"]): # "Downloaded" is shown as "False"
        PDFs_with_Wildcard=[x.replace('_','*') for x in PDFs]
        # for Repeat in range(5):
        #     print("Repeat "+str(Repeat)+" times to remove all adjancent wildcards.")
        #     PDFs_with_Wildcard=[x.replace('**','*') for x in PDFs_with_Wildcard] # Remove adjancent wildcards.
        RegexMatch=False
        if (List_Papers.iloc[i])["Name"]+".pdf" in PDFs:
        #     RegexMatch=True
        # else: # Regex match is very slow. Better do it after the first round.
        #     print("See if the file name "+(List_Papers.iloc[i])["Name"]+" contains underline. (Do regex match).")
        #     for Regex in PDFs_with_Wildcard:
        #         try:
        #             RegexMatch=re.match(Regex,(List_Papers.iloc[i])["Name"]+'.pdf')
        #             if RegexMatch:
        #                 break
        #         except:
        #             pass
        # if RegexMatch:
            List_Papers.iloc[i,List_Papers.columns.get_loc("PDF Name")]=(List_Papers.iloc[i])["Name"]+'.pdf'
            List_Papers.iloc[i,List_Papers.columns.get_loc("Downloaded")]=True
        # else: # DO NOT USE! Do not remove DataFrame indexes in a for loop because it will dynamically change the data frame size
        #     List_Papers.drop(index=i,inplace=True)
List_Papers=List_Papers.loc[List_Papers.Downloaded,:] # Remove not downloaded entries
List_Papers["Duplicated"]=List_Papers["PDF Name"].duplicated()
List_Papers["Duplicated_Count"]=[0]*len(List_Papers.index) # Create a new column in DataFrame with zeros. len(List_Papers.index) can also be List_Papers.shape[0] (shape returns a 2D-tuple with num of rows and num of columns)
List_Papers["Duplicated_First_Occurance"]=[-1]*len(List_Papers.index)
Downloaded_Papers=[]
# for dirpath, dirnames, filenames in os.walk("."):
#     for filename in [f for f in filenames if f.endswith(".pdf")]:
#         Downloaded_Papers=Downloaded_Papers+[filename]
# Papers_NoDuplicate=List_Papers[List_Papers["PDF Name"].isin(Downloaded_Papers)]
# DOIList=List_Papers["DOI"].tolist()
# DOIList=[x.lower() for x in DOIList]
for i in range(1,len(List_Papers.index)):
    # if (List_Papers.iloc[i])["DOI"] != "Unknown":
    #     # Check duplication by DOI.
    #     Previous_DOIs=DOIList[:i]
    #     if ((List_Papers.iloc[i])["DOI"]).lower() in Previous_DOIs:
    #         # List_Papers.iloc[i,List_Papers.columns.get_loc("DOI")]="Duplicated"
    #         # DOIList[i]="duplicated"
    #         List_Papers.iloc[i,List_Papers.columns.get_loc("Duplicated")]=True
    #         FileToDelete=[x for x in PDFs if (List_Papers.iloc[i])["PDF Name"] in x]
    #         for F in FileToDelete:
    #             try:
    #                 os.remove(F)
    #             except FileNotFoundError:
    #                 print("Already removed.")
    if list(List_Papers["Duplicated"])[i]:
        Current_Paper_Name=List_Papers.iloc[i,List_Papers.columns.get_loc("PDF Name")]
        Previous_Papers=list(List_Papers["PDF Name"])[slice(i)]
        List_Papers.iloc[i,List_Papers.columns.get_loc("Duplicated_Count")]=Previous_Papers.count(Current_Paper_Name)
        # try:
        Current_Paper_First_Occurance=Previous_Papers.index(Current_Paper_Name)
        # except ValueError: # It's not name but DOI that's duplicated
        #     Current_Paper_DOI=(List_Papers.iloc[i])["DOI"].lower()
        #     if Current_Paper_DOI != "unknown":
        #         Current_Paper_First_Occurance=(DOIList[slice(i)]).index(Current_Paper_DOI)
        List_Papers.iloc[i,List_Papers.columns.get_loc("Duplicated_First_Occurance")]=Current_Paper_First_Occurance
        Search_Terms=List_Papers.iloc[Current_Paper_First_Occurance,List_Papers.columns.get_loc("Search term")]+","+List_Papers.iloc[i,List_Papers.columns.get_loc("Search term")]
        List_Papers.iloc[Current_Paper_First_Occurance,List_Papers.columns.get_loc("Search term")]=Search_Terms
List_Papers=List_Papers.loc[~List_Papers["Duplicated"],:]
Search_Terms_Lists=[ [] for _ in range(len(List_Papers.index)) ]
List_Papers.drop(list(List_Papers.filter(regex='Duplicated')), axis=1, inplace=True)
for i in range(len(List_Papers.index)):
    Search_Terms_List=List_Papers.iloc[i,List_Papers.columns.get_loc("Search term")].split(",")
    Search_Terms_Lists[i]=Search_Terms_List
List_Papers["Search terms"]=Search_Terms_Lists
print("Total amount of papers downloaded as per CSV scan: "+str(len(List_Papers.index)))
with open("../4_Temp/Total_Numbers_unique_PDF_files.txt",'r',encoding='utf-8') as f:
    NumSavedPapers=int(f.read())
print("Total amount of papers downloaded as per file scan: "+str(NumSavedPapers))
if(NumSavedPapers != len(List_Papers.index)):
    print("The scanned total number of papers scanned from \"result.csv\" does not match the total PDF number.\nCheck what paper might be missing in the list.")
    with open("../4_Temp/Unique_PDF_files.txt","r",encoding="utf-8") as f:
        PDFFiles=f.read().splitlines()
    Downloaded_Papers_Not_in_List=[x for x in PDFFiles if x not in List_Papers['PDF Name'].tolist()]
    Papers_in_List_Not_Downloaded=[x for x in List_Papers['PDF Name'].tolist() if x not in PDFFiles]
    if len(Downloaded_Papers_Not_in_List) > 0:
        print("Downloaded papers not in the JSON list of papers: "+str(len(Downloaded_Papers_Not_in_List))+"\nYou need to parse these files manually by editing the CSV files.\n")
        Downloaded_Papers_Not_in_List_WithPath=[]
        import glob
        for Paper in Downloaded_Papers_Not_in_List:
            for F in glob.glob('../2_Papers/**/'+Paper,recursive=True):
                Downloaded_Papers_Not_in_List_WithPath.append(F)
        Downloaded_Papers_Not_in_List_WithPath.sort()
        with open("../4_Temp/Downloaded_Papers_Not_in_List.txt",'w',encoding='utf-8') as f:
            f.write('\n'.join(Downloaded_Papers_Not_in_List_WithPath))
        PapersNotInList={}
        for PaperPath in Downloaded_Papers_Not_in_List_WithPath:
            Path=os.path.basename(os.path.dirname(PaperPath))
            File=os.path.basename(PaperPath)
            if Path in PapersNotInList.keys():
                PapersNotInList[Path].append(File)
            else:
                PapersNotInList[Path]=[File]
        # for Path in PapersNotInList.keys():
        #     with open ('../2_Papers/'+Path+'/_Papers_Not_in_CSV_List.txt','w',encoding='utf-8') as f:
        #         f.write('\n'.join(PapersNotInList[Path]))
    else:
        try:
            os.remove("../4_Temp/Downloaded_Papers_Not_in_List.txt")
        except FileNotFoundError:
            pass
    if len(Papers_in_List_Not_Downloaded) > 0:
    #     with open("Papers_in_List_Not_Downloaded.txt",'w',encoding='utf-8') as f:
    #         f.write('\n'.join(Papers_in_List_Not_Downloaded))
        print("Entries in JSON that's not present: "+str(len(Papers_in_List_Not_Downloaded))+". These missing papers are possibly duplications with different name but same DOI/size, removed by the code in the next two sections.")
        List_Papers=List_Papers[~List_Papers["PDF Name"].isin(Papers_in_List_Not_Downloaded)]
print("\nTotal entries in list: "+str(len(List_Papers.index))+"\n")
List_Papers.to_json("../3_Lists/List_Papers.json",orient="records")
with open("../3_Lists/List_Papers.json",'r',encoding="utf-8") as fp:
    JSON_Data=json.load(fp)
    JSON_Data_Formatted=json.dumps(JSON_Data,indent=4)
with open("../3_Lists/List_Papers.json",'w',encoding="utf-8") as fp:
    fp.write(JSON_Data_Formatted)

After investigations, it seems that some papers are downloaded but their download status was still "false"
Try to parse those PDF files...
Total amount of papers downloaded as per CSV scan: 3598
Total amount of papers downloaded as per file scan: 3448
The scanned total number of papers scanned from "result.csv" does not match the total PDF number.
Check what paper might be missing in the list.
Entries in JSON that's not present: 150. These missing papers are possibly duplications with different name but same DOI/size, removed by the code in the next two sections.

Total entries in list: 3448



#### Remove duplicate file with possibly different names but same DOI
This section could be merged to above, but will significantly increase the time needed.

In [5]:
import pandas,json,os,glob
List_Papers=pandas.read_json("../3_Lists/List_Papers.json")
List_Papers_DOI=[x.lower() for x in List_Papers["DOI"].tolist()] # change to lower case to find DOI match
# print(len(List_Papers_DOI))
# Papers_woDOI=List_Papers[List_Papers.DOI == "Unknown"]
# Papers_woDOI.drop(columns=['DOI','Journal'],inplace=True)
# # Papers_WithDOI=List_Papers[List_Papers.DOI != "Unknown"]
# Papers_woDOI.to_json("../3_Lists/List_Papers_noDOI.json",orient="records")
# with open("../3_Lists/List_Papers_noDOI.json",'r',encoding="utf-8") as fp:
#     JSON_Data=json.load(fp)
#     JSON_Data_Formatted=json.dumps(JSON_Data,indent=4)
# with open("../3_Lists/List_Papers_noDOI.json",'w',encoding="utf-8") as fp:
#     fp.write(JSON_Data_Formatted)
DOIList=List_Papers["DOI"].tolist()
DOIList=[x.lower() for x in DOIList] # convert all letters to lower case for case insensitive matching
DOI_Repitition_First_Occurence=[-1]*len(DOIList)
for i in range(1,len(DOIList)):
    if DOIList[i] == "unknown":
        continue
    Previous_DOIs=DOIList[slice(i)]
    # print(Previous_DOIs)
    try:
        DOI_Repitition_First_Occurence[i]=Previous_DOIs.index(DOIList[i])
    except ValueError:
        pass
    if DOI_Repitition_First_Occurence[i] != -1:
        Search_Terms_To_Add=list(set((List_Papers.iloc[i])["Search terms"])-set((List_Papers.iloc[DOI_Repitition_First_Occurence[i]])["Search terms"]))
        if DOI_Repitition_First_Occurence[DOI_Repitition_First_Occurence[i]] == -1:
            print('Old: '+str(List_Papers.iloc[DOI_Repitition_First_Occurence[i],List_Papers.columns.get_loc("Search terms")]))
            List_Papers.iloc[DOI_Repitition_First_Occurence[i],List_Papers.columns.get_loc("Search terms")].extend(Search_Terms_To_Add)
            print('New: '+str(List_Papers.iloc[DOI_Repitition_First_Occurence[i],List_Papers.columns.get_loc("Search terms")])+'\n\n')
        else:
            raise Exception('ERROR: The supposed 1st location of the duplication is also a duplication')
        # List_Papers.iloc[i,List_Papers.columns.get_loc("DOI")]="Duplicated"
        # DOIList[i]="duplicated"
print("In total there are "+str(len([x for x in DOI_Repitition_First_Occurence if x != -1]))+" files with different name but duplicated DOI.")
with open("../4_Temp/PDF_files_with_FullPath.txt","r",encoding="utf-8") as f:
    PDFs=f.read().splitlines()
for i in range(len(DOI_Repitition_First_Occurence)):
    if DOI_Repitition_First_Occurence[i] != -1:
        Duplicated_PDF=(List_Papers.iloc[i])["PDF Name"]
        print(Duplicated_PDF+" is a duplicated file and shall be removed.")
        FileToDelete=[x for x in PDFs if Duplicated_PDF in x]
        for F in FileToDelete:
            try:
               os.remove(F)
            except FileNotFoundError:
                print("Already removed.")
        List_Papers.iloc[i,List_Papers.columns.get_loc("DOI")]="Duplicated"
print(str(list(List_Papers.DOI).count("Duplicated"))+" entries with duplicated result removed.")
List_Papers=List_Papers[ List_Papers.DOI != "Duplicated" ]
List_Papers.to_json("../3_Lists/List_Papers.json",orient="records")
with open("../3_Lists/List_Papers.json",'r',encoding="utf-8") as fp:
    JSON_Data=json.load(fp)
    JSON_Data_Formatted=json.dumps(JSON_Data,indent=4)
with open("../3_Lists/List_Papers.json",'w',encoding="utf-8") as fp:
    fp.write(JSON_Data_Formatted)
print("\nIn total there are "+str(len(List_Papers.index))+" papers listed.")
# print(str(DOI_Repitition_First_Occurence))

In total there are 0 files with different name but duplicated DOI.
0 entries with duplicated result removed.

In total there are 3448 papers listed.


#### Remove duplicate file with possibly different names but same data (check by file size)

In [14]:
# Check PDF files of each folder
import os,json,pandas,subprocess,time,msvcrt
# def Check_Duplications_Windows():
command="$Files=(Get-ChildItem \"..\\2_Papers\\*\\*.pdf\") ; $Groups = $Files | Group-Object Length ; $duplicates = $groups | Where-Object {$_.Count -gt 1} ; $hashtable=@{} ; foreach($d in $duplicates) { $hashtable.add([string]$d.name,[string[]]$d.group)} ; Convertto-json -input $hashtable | Out-File ..\\4_Temp\\Duplications_By_Size.json -Force"
process=subprocess.Popen(["powershell.exe","-Command",command],stdout=subprocess.PIPE)
# return
# Check_Duplications_Windows()
List_Papers=pandas.read_json("../3_Lists/List_Papers.json")
with open('../4_Temp/Duplications_By_Size.json','r',encoding='utf-16') as f: # JSON Files created by PowerShell are in UTF-16 format.
    SizeDuplications=json.load(f)
if len(SizeDuplications.keys()) == 0:
    raise Exception("No file duplication anymore.")
else:
    print("There are "+str(len(SizeDuplications.keys()))+" duplication cases found in total.\n")
Total_Num_Files_Need_To_Remove=0
for size in SizeDuplications.keys():
    Dups=SizeDuplications[size]
    Total_Num_Files_Need_To_Remove+=(len(Dups)-1)
    Dups_SearchTerm=[""]*len(Dups)
    Dups_PDFName=[""]*len(Dups)
    Dups_Bibtex=[False]*len(Dups)
    Dups_Index=[-1]*len(Dups)
    for i in range(len(Dups)):
        Dups_PDFName[i]=os.path.basename(Dups[i])
        Dups_SearchTerm[i]=os.path.basename(os.path.dirname(Dups[i]))
        File_Entry_in_List=(List_Papers[List_Papers["PDF Name"] == Dups_PDFName[i]]).index.values
        if len(File_Entry_in_List) != 1:
            raise Exception("The PDF file name is not unique!")
        Dups_Index[i]=File_Entry_in_List[0]
        Dups_Bibtex[i]=(List_Papers.iloc[Dups_Index[i]])["Bibtex"]
    # Delete the one without bibtex, which is probably the wrong one.
    # print(Dups_Bibtex)
    Manual_Parse_Needed=False
    if sum(Dups_Bibtex) > 0:
        for i in range(len(Dups)):
            if not Dups_Bibtex[i]:
                os.remove(Dups[i])
                # print('Removing file with wrong name: '+Dups_PDFName[i])
                List_Papers.iloc[Dups_Index[i],List_Papers.columns.get_loc("DOI")]="Duplicated"
        if sum(Dups_Bibtex) > 1:
            print("This duplication pair has MULTIPLE ones with Bibtex=true. Please parse manually:\n"+str(Dups)+'\n')
            Manual_Parse_Needed=True
    elif sum(Dups_Bibtex) == 0:
        print("This pair do not have any one with Bibtex=true. Please parse manually:\n"+str(Dups)+'\n')
        Manual_Parse_Needed=True
    if Manual_Parse_Needed:
        Prompt="\n"
        for i in range(len(Dups)):
            DupsAlreadyDeleted=False
            try:
                os.startfile(Dups[i])
            except FileNotFoundError:
                DupsAlreadyDeleted=True
                break
            Prompt = Prompt + str(i) + '. ' + Dups[i] +'\n'
        if not DupsAlreadyDeleted:
            ToKeep=-1
            while ToKeep not in range(len(Dups)) and ToKeep != 99 and ToKeep !=77:
                print("Which one paper of the duplication shall be KEPT? Give 99 to confirm that both files are not the same (Delete none). Give 77 to remove both files:"+Prompt)
                ToKeep = int(input(Prompt))
            if ToKeep != 99:
                for i in range(len(Dups)):
                    if i != ToKeep:
                        print(Dups[i]) 
                        try:
                            os.remove(Dups[i])
                        except PermissionError:
                            input("Close Adobe Acrobat before proceeding!")
                            os.remove(Dups[i])
                        List_Papers.iloc[Dups_Index[i],List_Papers.columns.get_loc("DOI")]="Duplicated"
            # timeout = 2
            # print("Press any key to end the process or the next paper will be opened within "+str(timeout)+" seconds.")
            # startTime = time.time()
            # inp = None
            # while time.time() - startTime <= timeout:
            #     if msvcrt.kbhit(): # if keyboard is hit
            #         inp = msvcrt.getch()
            #         break
            # if inp:
            #     break
            # else:
            print("\n----------------------\n")
Total_Num_Removed_Duplications=List_Papers["DOI"].to_list().count("Duplicated")
print(str(Total_Num_Files_Need_To_Remove)+" files have to be removed to eliminate duplications.")
print(str(Total_Num_Removed_Duplications)+" actually removed as of now.")
List_Papers=List_Papers[ List_Papers.DOI != "Duplicated" ]
print("\nTotal entries in list: "+str(len(List_Papers.index))+"\n")
List_Papers.to_json("../3_Lists/List_Papers.json",orient="records")
with open("../3_Lists/List_Papers.json",'r',encoding="utf-8") as fp:
    JSON_Data=json.load(fp)
    JSON_Data_Formatted=json.dumps(JSON_Data,indent=4)
with open("../3_Lists/List_Papers.json",'w',encoding="utf-8") as fp:
    fp.write(JSON_Data_Formatted)

There are 285 duplication cases found in total.

This duplication pair has MULTIPLE ones with Bibtex=true. Please parse manually:
['Z:\\2_Papers\\Friction stir welding Magnesium Automotive applications\\Friction stir welding of AZ31 magnesium alloy_ A review.pdf', 'Z:\\2_Papers\\Friction stir welding Magnesium Lightweight applications\\Friction stir welding of AZ31 magnesium alloy_ a review.pdf']



FileNotFoundError: [WinError 2] Das System kann die angegebene Datei nicht finden: 'Z:\\2_Papers\\Metal active gas welding Copper Automotive applications\\Evaluation of mechanical properties of dissimilar metal tube welded joints using inert gas welding.pdf'

#### Manually parse papers not included in CSV file:
This script will
- Open the file with the default PDF viewer
- Ask questions (User input: name, DOI, year, journal.)
- Add the results to CSV
- Wait for 3 seconds and open the next PDF file with the default PDF viewer

In [17]:
import glob,os,sys,pathlib,time,msvcrt
# from tkinter import *
if os.path.isfile('../4_Temp/Downloaded_Papers_Not_in_List.txt'):
    with open('../4_Temp/Downloaded_Papers_Not_in_List.txt','r',encoding='utf-8') as f:
        FilesToParse=f.read().splitlines()
    for File in FilesToParse:
        CurrentPath=os.path.dirname(File)
        Paper=os.path.basename(File)
        print("Manually parsing papers in folder \""+os.path.basename(CurrentPath)+"\"")
        try:
            os.startfile(File)
        except FileNotFoundError:
            print("File is removed. Skipping...")
        print("What's the name of this paper? If you leave this empty then the file name without the pdf extension will be used.\n")
        PaperName = input('Default name: '+pathlib.Path(Paper).stem).strip() or pathlib.Path(Paper).stem
        DOI = input("What's the DOI of this paper? ")
        Journal = input("On which journal is it published?")
        Year = input("Which year is the paper published? ")
        CSV_NewLine=PaperName+",,"+DOI+",True,"+Paper+","+Year+",,"+Journal+",True,,"
        with open(CurrentPath+"\\result.csv",'a+',encoding='utf-8') as f:
            filedata=f.read()
            if not filedata.endswith('\n'):
                CSV_NewLine='\n'+CSV_NewLine
            f.write(CSV_NewLine)
        timeout = 3
        print("Press any key to end the process or the next paper will be opened within "+str(timeout)+" seconds.")
        startTime = time.time()
        inp = None
        while time.time() - startTime <= timeout:
            if msvcrt.kbhit(): # if keyboard is hit
                inp = msvcrt.getch()
                break
        if inp:
            break
        else:
            print("\n----------------------\n")
    print("Parsing finished. Run the section \"List Papers by DOI\" again.")
else:
    print("All files parsed.")


Manually parsing papers in folder "Laser beam welding Magnesium Aerospace applications"
What's the name of this paper? If you leave this empty then the file name without the pdf extension will be used.



KeyboardInterrupt: Interrupted by user

#### Try to find papers published on potential predatory journals
Uses Beall's List as a reference.

https://beallslist.net/

https://beallslist.net/standalone-journals/

https://raw.githubusercontent.com/saketkc/ugc-predatory-journal-analysis/master/Beall_list_Jan2017.txt

In [4]:
# import wpparser
import requests
import json
list_url="https://beallslist.net/blog/wp-json/wp/v2/posts"
headers=headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
webpage=requests.get(list_url,headers=headers)
with open('../5_Predatory_Check/Bealls_List.xml','w',encoding='utf-8') as f:
    publisher_list=['<root>']
    webpage_list=webpage.text.splitlines()
    for x in webpage_list:
        if x.startswith('<li><a href='):
            publisher_list=publisher_list+[x]
    publisher_list=publisher_list+['</root>']
    f.write("\n".join(publisher_list))
journal_list=(requests.get('https://raw.githubusercontent.com/saketkc/ugc-predatory-journal-analysis/master/Beall_list_Jan2017.txt')).text
with open('../5_Predatory_Check/Predatory_Journals.txt','w',encoding='utf-8') as f:
    f.write(journal_list)

Compare papers from "List_Papers.json" with Journal_List

In [13]:
import json
import pandas
with open("../3_Lists/List_Papers.json",'r',encoding="utf-8") as f:
    Data=json.load(f)
List_Papers=pandas.json_normalize(Data)
List_Papers["Predatory"]=[False]*len(List_Papers.index)
with open('../5_Predatory_Check/Predatory_Journals.txt','r',encoding='utf-8') as f:
    Predatory_Journals=f.read().splitlines()
# Remove brackets (abbreviations) in the list.
import re
Predatory_Journals=[re.sub(r'\(.*?\)','',x) for x in Predatory_Journals] # Remove abbreviations in brackets
Predatory_Journals=[x.strip() for x in Predatory_Journals] # Remove ending space.
Num_Predatory_Papers=0
for i in range(len(List_Papers.index)):
    if (List_Papers.iloc[i])["Journal"] in Predatory_Journals:
        List_Papers.iloc[i,List_Papers.columns.get_loc("Predatory")]=True
        Num_Predatory_Papers+=1
print("Found "+str(Num_Predatory_Papers)+" paper(s) from predatory journals:")
for i in range(len(List_Papers.index)):
    if (List_Papers.iloc[i])["Predatory"]==True:
        print((List_Papers.iloc[i])["Name"]+", DOI: "+(List_Papers.iloc[i])["DOI"])
List_Papers.to_json("../3_Lists/List_Papers.json",orient="records")
import json
with open("../3_Lists/List_Papers.json",'r',encoding="utf-8") as fp:
    JSON_Data=json.load(fp)
    JSON_Data_Formatted=json.dumps(JSON_Data,indent=4)
with open("../3_Lists/List_Papers.json",'w',encoding="utf-8") as fp:
    fp.write(JSON_Data_Formatted)

Found 3 paper(s) from predatory journals:
Microstructural characteristics on bead on plate welding of AISI 904 L super austenitic stainless steel using gas metal arc welding process, DOI: 10.4314/ijest.v2i6.63710
Effect of process parameters of tungsten inert gas welding on the weld quality of austenitic stainless steel, DOI: 10.22214/ijraset.2018.1388
A review of Pulse current gas tungsten arc welding for different alloys, DOI: 10.51976/ijari.111311
